# Investigate Titanic Survivor

<img src='img/nytimes.jpg'>

This is a final project from online class in Udacity, [UD170 Intro to Data Analyst](https://classroom.udacity.com/courses/ud170/) to analyze titanic survivor by using [Titanic dataset](https://www.kaggle.com/c/titanic) on Kaggle Competition. This project focus on doing Exploratory Data Analysis (EDA) to extract insight on the dataset.

# <a name='Definition'>Problem Definition</a>

### Objective
- What factors made people more likely to survive?

### Description
>The sinking of the RMS Titanic is one of the most infamous shipwrecks in history.  On April 15, 1912, during her maiden voyage, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 passengers and crew. This sensational tragedy shocked the international community and led to better safety regulations for ships.

>One of the reasons that the shipwreck led to such loss of life was that there were not enough lifeboats for the passengers and crew. Although there was some element of luck involved in surviving the sinking, some groups of people were more likely to survive than others, such as women, children, and the upper-class.

## Import Libraries
We need to import python libraries that containing the necessary functionality we will need

In [1]:
# data analysis and wrangling
import pandas as pd
import numpy as np

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
% matplotlib inline

## Acquire Data

In [2]:
dataset = pd.read_csv('data/titanic_data.csv')
dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Analyze by describing data

### Which features are available in the dataset?

Here's the feature description:

- <b>`Survived`</b>: Outcome of survival
- <b>`Pclass`</b>: Socio-economic class
- <b>`Name`</b>: Name of passenger
- <b>`Sex`</b>: Sex of the passenger
- <b>`Age`</b>: Age of the passenger
- <b>`SibSp`</b>: Number of siblings and spouses of the passenger aboard
- <b>`Parch`</b>: Number of parents and children of the passenger aboard
- <b>`Ticket`</b>: Ticket number of the passenger
- <b>`Fare`</b>: Fare paid by the passenger
- <b>`Cabin`</b>: Cabin number of the passenger
- <b>`Embarked`</b>: Port of embarkation of the passenger 

For more information, you can see [this documentation.](https://www.kaggle.com/c/titanic/data)

### Which features are categorical?
Categorical represent types of data which may be divided into group. Example of categorical group are race, sex, age group, and educational level. Althought the latter two group may also be considered in a numerical manner, it's often more informative to categorize into small number of groups.
- `Survived` (True or False / 1 or 0)
- `Sex` (Male, Female) 
- `Embarked` (C, S, Q)
- `Pclass` (1, 2, 3)

### Which features are numerical?
Numerical represent types of data which can be measurement, such as a person's height, weight, IQ, or blood pressure. It can be discrete or continous.
- `Age` (continous)
- `Fare` (continous)
- `Sibsp` (discrete)
- `Parch` (discrete)

### Which features are mixed data types?
Numerical, alphanumeric data within same feature. These are candidates for correcting.
- `Ticket` is a mix of numeric and alphanumeric
- `Cabin` is alphanumeric

### Which features may contain errors or typos?
This is harder to review for a large dataset, however reviewing a few samples from a smaller dataset may just tell us outright, which features may require correcting.
- `Name` feature may contain error of typos as there are several ways used to describe a name including titles, round brackets, and quotes used for alternative or short names

### Which features contain blank, null or empty values?
These will require correcting before used for analyzing.
- `Cabin` : 687 records
- `Age` : 177 records
- `Embarked` : 2 records

In [3]:
total_null = dataset.isnull().sum()
percent_null = (round(total_null/dataset.isnull().count()*100, 2))
missing_data = pd.concat([total_null, percent_null], axis=1, keys=['Total', '%']).sort_values('%', ascending=False)
missing_data.head()

,Total,%
Cabin,687,77.10
Age,177,19.87
Embarked,2,0.22
PassengerId,0,0.00
Survived,0,0.00


`Embarked` featue has only 2 missing values, which can easily to handled. To deal with `Age` which has 117 missing feature, it'll be much more tricky. The `Cabin` feature needs further investigation, but it looks like that we might want to drop from the dataset since 77% of missing value

### What is the distribution of numerical feature values across the samples?
This helps us determine, among other early insights, how representative is the dataset of the actual problem domain.
- Total samples are 891 or 40% of actual number of passenger on board the titanic (2,224)
- There are 38% survived representasive of actual survival rate at 32%
- Few elderly passenger (<1) within age range 65-80
- Fares varied significantly with few passengers (<1%) paying as high as $512.
- Most passengers (75%) did not travel with parent and child
- Nearly 33% passengers had siblings and/or spouse aboard

In [4]:
dataset.describe()
# age and fare : percentiles=[.1, .2, .3, .4, .5, .6, .7, .8, .9, .99]
# Parch : percentiles=[.75]
# SibSp : percentiles=[.68]

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


### What is the distribution of categorical features?
This helps us determine, among other early insights, how representative is the dataset of the actual problem domain.
- All name of passengers are unique (count=891)
- `Sex` feature have two possibles values with male is the most passengers (count=577/891)
- `Ticket` has high ratio (22%) of duplicate values
- `Cabin` also has duplicate values. That might be several passenger shared a cabin
- There are three possible of `Embarked` values (S=Southampton, Q=Queenstown, C=Cherbourg)

In [5]:
dataset.describe(include=['O'])

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Sage, Mr. George John Jr",male,CA. 2343,B96 B98,S
freq,1,577,7,4,644
